In [19]:
import sys
import os
import glob
import shutil

from matplotlib import pyplot as plt
import matplotlib
from tqdm import tqdm
import numpy as np
import cv2 as cv2
from PIL import Image

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F
from torchvision.transforms import Compose, RandomHorizontalFlip, \
                RandomVerticalFlip, ColorJitter, RandomRotation, GaussianBlur, RandomCrop, RandomPerspective, RandomAffine
from torch.utils.tensorboard import SummaryWriter



sys.path.append('C:\\Users\\LuCo\\Documents\\repos\\Depth-Anything-V2')
from depth_anything_v2.dpt import DepthAnythingV2
from metric_depth.dataset.transform import Resize, NormalizeImage, PrepareForNet, Crop


In [20]:
# Global Parameters
# ------------------------------

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DATA_DIR = os.path.join(os.getcwd(), 'data', "train", "train")
CHECKPOINT_DIR = os.path.join(os.getcwd(), 'checkpoints')
BEST_CHECKPOINT_PATH = os.path.join(CHECKPOINT_DIR, 'best_depth_anything_v2.pth')
BACKUP_CHECKPOINT_PATH = os.path.join(CHECKPOINT_DIR, 'depth_anything_v2.pth')
PATIENCE_CHECKPOINT_PATH = os.path.join(CHECKPOINT_DIR, 'patience_depth_anything_v2_vitg.pth')
ENCODER = 'vitl'
NUM_EPOCHS = 12
LOG_DIR = os.path.join(os.getcwd(), 'logs')
RESULT_DIR = os.path.join(os.getcwd(), 'results')

In [21]:
class DepthDataset(Dataset):
    def __init__(self, root, mode='train', size = (518, 518)):
        self.root = root
        self.mode = mode
        self.size = size

        self.rgb_paths = sorted(glob.glob(os.path.join(root, '*_rgb.png')))

        if self.mode == 'train':
            self.depth_paths = [p.replace('_rgb.png', '_depth.npy') for p in self.rgb_paths]

        net_w, net_h = size


        self.transform = Compose([
            Resize(width=net_w, height=net_h, 
                   resize_target=True if mode == 'train' else False, 
                   keep_aspect_ratio=True, 
                   ensure_multiple_of=14, 
                   resize_method="lower_bound",
                   image_interpolation_method=cv2.INTER_CUBIC),
            NormalizeImage(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            PrepareForNet(),
        ] + ([Crop(size[0])] if mode == 'train' else []))
        

    def __len__(self):
        return len(self.rgb_paths)
    

    def horizontal_flip(self, image, depth, p=0.3):
        if np.random.rand() < p:
            image = np.fliplr(image).copy()
            depth = np.fliplr(depth).copy()
        return image, depth
    
    def brightness_jitter(self, image, p = 0.25):
        if np.random.rand() < p:
            hsv = cv2.cvtColor((image * 255).astype(np.uint8), cv2.COLOR_RGB2HSV)
            hsv = np.array(hsv, dtype=np.float32)
            hsv[..., 2] *= 0.8 + 0.4 * np.random.rand()
            hsv[..., 2] = np.clip(hsv[..., 2], 0, 255)
            rgb = cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2RGB)
            image = rgb.astype(np.float32) / 255.0
            
        return image

    def __getitem__(self, idx):
        # Load RGB image
        rgb = cv2.imread(self.rgb_paths[idx])
        rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB) / 255.0
        
        # Load depth map
        if self.mode == 'train':
            depth = np.load(self.depth_paths[idx])

        # Apply data augmentation (e.g. horizontal flip)
        if self.mode == 'train':
            rgb, depth = self.horizontal_flip(rgb, depth, p=0.1)
            rgb = self.brightness_jitter(rgb, p=0.1)

            sample = self.transform({'image': rgb, 'depth': depth})
        
        if self.mode == 'test':
            sample = self.transform({'image': rgb})
            sample['image'] = torch.from_numpy(sample['image'])

        if self.mode == 'test':
            sample['filenames'] = self.rgb_paths[idx]
            
        if self.mode == 'train':
            sample['depth'] = torch.from_numpy(sample['depth'])

            sample['valid_mask'] = (torch.isnan(sample['depth']) == 0)
            sample['depth'][sample['valid_mask'] == 0] = 0

        return sample

In [22]:
class ScaleInvariantRMSELoss(nn.Module):
    def __init__(self):
        super(ScaleInvariantRMSELoss, self).__init__()

    def forward(self, pred, target):
        pred = pred.squeeze()
        target = target.squeeze()

        pred_flat = pred.view(pred.size(0), -1)
        target_flat = target.view(target.size(0), -1)

        log_pred = torch.log(pred_flat + 1e-8)
        log_target = torch.log(target_flat + 1e-8)

        delta = log_target - log_pred

        alpha = delta.mean(dim=1, keepdim=True)

        loss = ((-delta + alpha) ** 2).mean(dim=1).sqrt()

        return loss.mean()

In [ ]:
    def compute_losses(self, inputs, outputs):
        """Compute the reprojection and smoothness losses for a minibatch
        """
        losses = {}
        total_loss = 0

        for scale in self.opt.scales:
            loss = 0

            disp = outputs[("disp", scale)]
            color = inputs[("color", 0, scale)]
            target = inputs[("color", 0, scale)]

            for frame_id in self.opt.frame_ids[1:]:
                pred = outputs[("color", frame_id, scale)]
                pred_new = outputs[("color_new", frame_id, scale)]

                outputs[("reprojection_losses_vitual", frame_id, scale)] = self.compute_reprojection_loss(pred, target)
                loss += outputs[("reprojection_losses_vitual", frame_id, scale)].mean()

                outputs[("reprojection_losses_new", frame_id, scale)] = self.compute_reprojection_loss(pred_new, target)
                loss += outputs[("reprojection_losses_new", frame_id, scale)].mean()


            if self.opt.disable_plane_smoothness:
                mean_disp = disp.mean(2, True).mean(3, True)
                norm_disp = disp / (mean_disp + 1e-7)
                smooth_loss = get_smooth_loss(norm_disp, color)
            else:
                mean_coeff = outputs[("coeff", scale)].abs().mean(2, True).mean(3, True)

                norm_coeff = outputs[("coeff", scale)] / (mean_coeff + 1e-7)
                smooth_loss = get_smooth_loss(norm_coeff, color)

            loss += self.opt.smoothness_weight / (2 ** scale) * smooth_loss
            losses["smooth_loss/{}".format(scale)] = smooth_loss

            point3D = outputs[("cam_points", 0, scale)][:, :3, ...]
            mean_depth = outputs[("depth", 0, scale)].mean(2, True).mean(3)
            norm_point3D = point3D/(mean_depth + 1e-7)

            if not self.opt.disable_plane_regularization:
                key = ("plane_keysets", 0, scale)
                if key in inputs:
                    plane_loss = get_plane_loss(inputs[key], norm_point3D)
                    loss += self.opt.plane_weight * plane_loss
                    losses["plane_loss/{}".format(scale)] = plane_loss
                else:
                    print(f"[Warning] Missing key {key} in inputs during compute_losses.")

            if not self.opt.disable_line_regularization:
                key = ("line_keysets", 0, scale)
                if key in inputs:
                    line_loss = get_line_loss(inputs[key], norm_point3D)
                    loss += self.opt.line_weight * line_loss
                    losses["line_loss/{}".format(scale)] = line_loss
                else:
                    print(f"[Warning] Missing key {key} in inputs during compute_losses.")

            losses["loss/{}".format(scale)] = loss
            total_loss += loss

        total_loss /= self.num_scales
        losses["loss"] = total_loss
        return losses

In [23]:
class DepthGradientLoss(nn.Module):
    def __init__(self, edge_aware=True):
        super(DepthGradientLoss, self).__init__()
        self.edge_aware = edge_aware

    def forward(self, pred, image):
        pred_dx = torch.abs(pred[ :, :, :-1] - pred[ :, :, 1:])
        pred_dy = torch.abs(pred[ :, :-1, :] - pred[ :, 1:, :])

        if self.edge_aware:
            image_dx = torch.mean(torch.abs(image[ :, :, :-1] - image[ :, :, 1:]), 1, keepdim=True)
            image_dy = torch.mean(torch.abs(image[ :, :-1, :] - image[ :, 1:, :]), 1, keepdim=True)
            weight_x = torch.exp(-image_dx)
            weight_y = torch.exp(-image_dy)
            pred_dx *= weight_x
            pred_dy *= weight_y

        return (pred_dx.mean() + pred_dy.mean())


In [24]:
if not os.path.exists(LOG_DIR):
    os.makedirs(LOG_DIR)
# get current time for logging
from datetime import datetime
log_path = os.path.join(LOG_DIR, datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + '_train.log')
writer = SummaryWriter(log_dir=log_path, comment='No Validation Set')

In [25]:
import gc
torch.cuda.empty_cache()
gc.collect()

1149

In [ ]:
def train():
    dataset = DepthDataset(DATA_DIR, mode='train')
    val_size = int(len(dataset) * 0.1)
    train_size = len(dataset) - val_size

    print(dataset.root)
    print(len(dataset), train_size, val_size)
    train_set, val_set = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_set, batch_size=8, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_set, batch_size=8, shuffle=False, num_workers=0)

    model_configs = {
    'vits': {'encoder': 'vits', 'features': 64, 'out_channels': [48, 96, 192, 384]},
    'vitb': {'encoder': 'vitb', 'features': 128, 'out_channels': [96, 192, 384, 768]},
    'vitl': {'encoder': 'vitl', 'features': 256, 'out_channels': [256, 512, 1024, 1024]
             },
    'vitg': {'encoder': 'vitg', 'features': 384, 'out_channels': [1536, 1536, 1536, 1536]}
    }

    # Load the model with the specified encoder
    model = DepthAnythingV2(**model_configs[ENCODER])

    for param in model.pretrained.parameters():
        param.requires_grad = False
        

        
    for param in model.depth_head.parameters():
        param.requires_grad = True


    # Load the pretrained weights
    model.pretrained.load_state_dict(torch.load(os.path.join(CHECKPOINT_DIR, 'epoch_depth_anything_v2_checkpoint.pth')), strict=False)
    model = model.to(DEVICE)
    loss_fn_si_RMSE = ScaleInvariantRMSELoss()
    #loss_fn_gradient = DepthGradientLoss(edge_aware=True)

    optimizer = torch.optim.AdamW([
        {"params" : model.pretrained.parameters(), "lr" : 1e-5},
        {"params" : model.depth_head.parameters(), "lr" : 1e-5}], weight_decay=1e-4)

    best_val_loss = 0.22


    
    patience = 5
    patience_counter = 0

    for epoch in range(NUM_EPOCHS):
        print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
        model.train()
        train_loss = 0.0

        for i, sample in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1} [Train]")):
            optimizer.zero_grad()
            
            rgb = sample['image']
            depth = sample['depth']

            rgb = rgb.to(DEVICE)
            depth = depth.to(DEVICE)

            # print(f"Model device: {next(model.parameters()).device}")
            # print(f"RGB device: {rgb.device}, Depth device: {depth.device}")

            optimizer.zero_grad()
            pred = model(rgb)

            #loss_fn_gradient_ = loss_fn_gradient(pred, depth)
            loss_fn_si_RMSE_ = loss_fn_si_RMSE(pred, depth)
            #loss_l1 = torch.mean(torch.abs(torch.log(pred + 1e-6) - torch.log(depth + 1e-6)))

            loss = loss_fn_si_RMSE_ # + 0.01 * loss_l1 + 0.25 * loss_fn_gradient_

            loss.backward()
            optimizer.step()

            # logging the loss
            if i % 50 == 0:
                #writer.add_scalar('Loss/train', loss.detach().item(), epoch * len(train_loader) + i)
                writer.add_scalar('Loss/train_si_RMSE', loss_fn_si_RMSE_.detach().item(), epoch * len(train_loader) + i)
                #writer.add_scalar('Loss/train_gradient', loss_fn_gradient_.item(), epoch * len(train_loader) + i)
                #writer.add_scalar('Loss/train_l1', loss_l1.item(), epoch * len(train_loader) + i)
            train_loss += loss.item()

        train_loss /= len(train_loader)
        print(f"Train Loss: {train_loss:.4f}")
        writer.add_scalar('Loss/Train_Epoch', train_loss, epoch)

        #Validation
        model.eval()
        val_loss = 0.0
        
        with torch.no_grad():
            for sample in tqdm(val_loader, desc=f"Epoch {epoch+1} [Validation]"):
                rgb = sample['image']
                depth = sample['depth']
                rgb = rgb.to(DEVICE)
                depth = depth.to(DEVICE)

                pred = model(rgb)
                val_loss_ = loss_fn_si_RMSE(pred, depth)
                val_loss += val_loss_.item()
        val_loss /= len(val_loader)
        print(f"Validation Loss: {val_loss:.4f}")
        writer.add_scalar('Loss/Validation_Epoch', val_loss, epoch)
        
        # Save the model if validation loss improves
        if not os.path.exists(CHECKPOINT_DIR):
            os.makedirs(CHECKPOINT_DIR)
        
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            # Move the previous best checkpoint to a backup location
            if os.path.exists(BEST_CHECKPOINT_PATH):
                shutil.move(BEST_CHECKPOINT_PATH, BACKUP_CHECKPOINT_PATH)

            torch.save(model.state_dict(), BEST_CHECKPOINT_PATH)
            print(f"Model saved to {BEST_CHECKPOINT_PATH}")
            print(f"Previous best model moved to {BACKUP_CHECKPOINT_PATH}")
        else:
            patience_counter += 1
            print(f"Patience counter: {patience_counter}/{patience}")
            torch.save(model.state_dict(), PATIENCE_CHECKPOINT_PATH)
        
        
        # Save the model every epoch
        torch.save(model.state_dict(), os.path.join(CHECKPOINT_DIR, f"epoch_depth_anything_v2_checkpoint.pth"))

        # Save a sample image and depth map for visualization

        # Load a sample from the training set
        rgb = cv2.imread(os.path.join(DATA_DIR, "sample_000000_rgb.png"))
        gt = np.load(os.path.join(DATA_DIR, "sample_000000_depth.npy"))

        # set the model to evaluation mode
        with torch.no_grad():
            pred = model.infer_image(rgb)
        d_min = np.min(pred)
        d_max = np.max(pred)

        depth_vis = (pred - d_min) / (d_max - d_min + 1e-6)
        
        cmap = matplotlib.colormaps.get_cmap('plasma')
        
        plt.figure(figsize=(15, 5))

        plt.subplot(1, 3, 1)
        plt.title('Input Image')
        plt.axis('off')
        plt.imshow(rgb)

        plt.subplot(1, 3, 2)
        plt.title('Predicted Depth Map')
        plt.axis('off')



        plt.imshow(depth_vis, cmap=cmap)

        plt.subplot(1, 3, 3)
        plt.title('Ground Truth')
        plt.axis('off')
        plt.imshow(gt, cmap=cmap)
        plt.savefig(os.path.join(RESULT_DIR, f"epoch_{epoch+1}_sample.png"))
        plt.close()

        if patience_counter >= patience:
            print("Early stopping triggered.")
            torch.save(model.state_dict(), PATIENCE_CHECKPOINT_PATH)
            break





train()

c:\Users\LuCo\Documents\repos\DeepAnything\DepthAnything\data\train\train
23971 21574 2397
Epoch 1/12


Epoch 1 [Train]: 100%|██████████| 2697/2697 [4:16:47<00:00,  5.71s/it]  


Train Loss: 0.3536


Epoch 1 [Validation]: 100%|██████████| 300/300 [07:36<00:00,  1.52s/it]


Validation Loss: 0.3576
Patience counter: 1/5
Epoch 2/12


...

Epoch 3 [Validation]: 100%|██████████| 300/300 [07:39<00:00,  1.53s/it]


Validation Loss: 0.3575
Patience counter: 3/5
Epoch 4/12


Epoch 4 [Train]:   1%|          | 26/2697 [02:27<4:13:01,  5.68s/it]


KeyboardInterrupt: 

: 

In [ ]:
def generate_test_predictions(model, test_loader, device):
    """Generate predictions for the test set without ground truth"""
    model.eval()
    
    predictions_dir = os.path.join(RESULT_DIR, 'test_predictions')
    # Ensure predictions directory exists
    if not os.path.exists(predictions_dir):
        os.makedirs(predictions_dir)
    
    with torch.no_grad():
        for sample in tqdm(test_loader, desc="Generating Test Predictions"):
            inputs = sample['image']
            filenames = sample['filenames']
            
            inputs = inputs.to(device)
            batch_size = inputs.size(0)
            
            # Forward pass
            outputs = model(inputs)
            
            # Ensure outputs have the correct shape before resizing
            if outputs.dim() == 3:  # If outputs have shape (N, H, W)
                outputs = outputs.unsqueeze(1)  # Add a channel dimension to make it (N, C, H, W)

            # Resize outputs to match original input dimensions (426x560)
            outputs = nn.functional.interpolate(
                outputs,
                size=(426, 560),  # Original input dimensions
                mode='bilinear',
                align_corners=True
            )
            
            # Save all test predictions
            for i in range(batch_size):
                # Get filename without extension
                filename = filenames[i].split('\\')[-1].replace('_rgb.png', '_depth.npy')
                
                # Save depth map prediction as numpy array
                depth_pred = outputs[i].cpu().squeeze().numpy()
                np.save(os.path.join(predictions_dir, f"{filename}"), depth_pred)
            
            # Clean up memory
            del inputs, outputs
        
        # Clear cache after test predictions
        torch.cuda.empty_cache()

TESTDIR = os.path.join(os.getcwd(), 'data', "test", "test")
test_dataset = DepthDataset(TESTDIR, mode='test')
test_loader = DataLoader(test_dataset, batch_size=5, shuffle=False, num_workers=0)

model_configs = {
    'vits': {'encoder': 'vits', 'features': 64, 'out_channels': [48, 96, 192, 384]},
    'vitb': {'encoder': 'vitb', 'features': 128, 'out_channels': [96, 192, 384, 768]},
    'vitl': {'encoder': 'vitl', 'features': 256, 'out_channels': [256, 512, 1024, 1024]},
    'vitg': {'encoder': 'vitg', 'features': 384, 'out_channels': [1536, 1536, 1536, 1536]}
}

model = DepthAnythingV2(**model_configs[ENCODER])
model.load_state_dict(torch.load(os.path.join(BEST_CHECKPOINT_PATH), map_location=DEVICE), strict=False)
model = model.to(DEVICE)
generate_test_predictions(model, test_loader, DEVICE)

Generating Test Predictions: 100%|██████████| 130/130 [02:07<00:00,  1.02it/s]


In [ ]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), 'src'))
from src.create_prediction_csv import get_submission

get_submission()

c:\Users\LuCo\Documents\repos\DeepAnything\DepthAnything
c:\Users\LuCo\Documents\repos\DeepAnything\DepthAnything\results\test_predictions
c:\Users\LuCo\Documents\repos\DeepAnything\DepthAnything\data\test_list.txt
c:\Users\LuCo\Documents\repos\DeepAnything\DepthAnything\results\csv\FT_DepthAnything_Best.csv


Processing depth maps: 100%|██████████| 650/650 [01:46<00:00,  6.09it/s]


CSV file saved to: c:\Users\LuCo\Documents\repos\DeepAnything\DepthAnything\results\csv\FT_DepthAnything_Best.csv
Shape of the CSV: (650, 2)
